In [2]:
# 설치 후 버전확인

import implicit
import scipy
import pandas

print(implicit.__version__)
print(scipy.__version__)
print(pandas.__version__)

0.7.2
1.16.0
2.3.1


In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import coo_matrix, csr_matrix
from implicit.als import AlternatingLeastSquares

In [4]:
# 데이터 준비
data = pd.DataFrame({
    'user_id': [0, 1, 1, 2, 3, 3],
    'item_id': [101, 101, 102, 103, 102, 104],
    'purchase_count': [1, 1, 2, 1, 1, 1]
})

In [6]:
# 인코딩

'''
추천 시스템에서 user_id나 item_id는 반드시 "0부터 시작하는 정수(index)"여야 하기 때문에 라벨인코딩을 진행
implicit 라이브러리에서 사용하는 희소행렬(sparse matrix)은
행과 열을 정수 인덱스 기반으로 다루기 때문에, user_id, item_id가 문자나 큰 숫자(예: 101, 9999 등)면 안 됩니다.
'''

user_enc = LabelEncoder()
item_enc = LabelEncoder()
data['user_idx'] = user_enc.fit_transform(data['user_id']) # 'fit():고유값들을 숫자로 매핑'와 'transform():실제 값을 그 숫자 매핑'으로 전환 동시 진행
data['item_idx'] = item_enc.fit_transform(data['item_id'])

In [7]:
# 데이터 확인
data

,user_id,item_id,purchase_count,user_idx,item_idx
0,0,101,1,0,0
1,1,101,1,1,0
2,1,102,2,1,1
3,2,103,1,2,2
4,3,102,1,3,1
5,3,104,1,3,3


In [16]:
# 희소행렬 (user x item)
'''
사용자-상품간의 구매 데이터를 행렬로 나타낸다.
coo_matrix((데이터값, (행 인덱스, 열 인덱스)))

즉, user_idx는 행이 되고 item_idx열이된다
대부분의 값이 0이므로 메모리 절약을 위해 희소행렬(coo_matrix)사용 -> 0은 저장하지 않고 값이 있는 위치만 기록
'''
matrix = coo_matrix((data['purchase_count'], (data['user_idx'], data['item_idx'])))

print(matrix.shape)  #  희소행렬 크기 확인 -> 예: (4, 4) → 4명의 유저, 4개의 아이템
print(matrix.toarray()) # 희소행렬 구조 확인
print(matrix.T.toarray()) # 행렬 전환(전치) 구조 확인(item_idx가 행, user_idx가 열)

(4, 4)
[[1 0 0 0]
 [1 2 0 0]
 [0 0 1 0]
 [0 1 0 1]]
[[1 1 0 0]
 [0 2 0 1]
 [0 0 1 0]
 [0 0 0 1]]


In [14]:
# ALS 학습
'''
ALS (Alternating Least Squares)는 추천 시스템에서 많이 쓰이는 행렬 분해(Matrix Factorization) 알고리즘
목표: 유저-아이템 간 선호 점수(predict score)를 예측

factors=10
사용자와 아이템의 잠재 요인(latent factor) 수(유저/아이템을 10차원 벡터로 표현)

iterations=15
ALS 학습 반복 횟수(15번 반복하면서 점점 정확해짐)
'''
model = AlternatingLeastSquares(factors=10, iterations=15)

C:\Users\EZENIC-213T\anaconda3\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()


In [15]:
'''
(user x item) 희소행렬 → 전치 → (item x user) 형태로 학습

matrix.T: implicit에서는 아이템-유저 형태로 학습해야 함(행렬전환)
.tocsr(): ALS 모델 학습은 CSR 형식을 요구함 (COO → CSR로 변환)

*CSR란? 
행렬에 0값이 많아서 전부 다 저장하면 메모리 낭비
예: 유저 10,000명 × 아이템 50,000개 → 대부분은 구매하지 않았음 (0)
-> 따라서 0이 아닌 값들만 저장하는 방법이 필요. 그게 CSR

원래 행렬
[[1 0 0 0]
 [1 2 0 0]
 [0 0 1 0]
 [0 1 0 1]]

CSR 형식은 이렇게 저장
data    = [1, 1, 2, 1, 1, 1] # 실제 값
indices = [0, 0, 1, 2, 1, 3] # 열 인덱스
indptr  = [0, 1, 3, 4, 6] # 행마다 어디서 시작하는지 표시

*참고
indptr 자세히 풀이
행 0 → data[0]부터 시작 → 1개 값 있음 → 끝: 1
행 1 → data[1] ~ data[2] → 2개 → 끝: 3
행 2 → data[3] → 1개 → 끝: 4
행 3 → data[4] ~ data[5] → 2개 → 끝: 6

그래서 indptr = [0, 1, 3, 4, 6] (행이 4개니까 원소가 5개)
'''

model.fit(matrix.T.tocsr())

  0%|          | 0/15 [00:00<?, ?it/s]